# OpenStreetMap Report 
### by Marty VanHoof

OpenStreetMap (OSM) is an open source project that volunteers work on in order to create a free editable map of the world.  It was created by Steve Coast in the UK in 2004, and is used by many organizations globally, including Craigslist, MapQuest, JMP statistical software, Red Cross, and many others:  http://wiki.openstreetmap.org/wiki/About_OpenStreetMap

The data is stored in XML format.  Unfortunately, since the data is human entered, the data quality can often be an issue.  The purpose of this project is to examine, audit, and clean some of this data programmatically.  I do this by first parsing, auditing, and cleaning the data using Python's XML ElementTree module, and then converting it from XML to CSV format.  Once the data is in CSV format, it can then be uploaded to a database making it convenient to run SQL queries to answer a number of questions about my chosen map area.

### The Map

The map I chose to work on is the region where I grew up:  Clarington, Ontario, Canada.  Clarington is a geographical region and a municipality in Ontario, Canada that consists of many smaller towns, villages, hamlets, etc.  In fact, I chose to download a bigger region than Clarington because I needed my dataset to be at least 50 MB in size.  I downloaded [this region](http://www.openstreetmap.org/export#map=11/43.9728/-78.6528) by using the "Overpass API" option, which downloads the entire area displayed in the box.

## Section 1:  Some Problems Encountered in the Map

There were various issues that I found, including:

- Extra whitespace in street names, e.g. "Concession Road  &nbsp;10" versus "Concession Road 10"
- Street name abbreviations such as Ave, St, Rd, etc.  Also, some abbreviations such as Rd E, which should be Road East
- Badly formatted postal codes: &nbsp; In Canada, postal codes are formatted with capital letters but my OSM data contained some postal codes with lower case letters such as "l1c 3k5", which should be "L1C 3K5".  Furthermore, some postal codes have extra characters or no whitespace.
- Some typos in city/town names, e.g. "Bowmanville" mistakenly spelled as "Bowmanwille".  Also, some inconsistency with city/town names such as "City of Oshawa" versus "Oshawa". 
- An ``` 'addr:housenumber' ``` tag with a missing ``` 'v' ``` value.

### Street Name Issues

To clean the street names, I modified the ` 'update_street_name()' ` function from the relevant portion of Udacity's [data wrangling course](https://classroom.udacity.com/nanodegrees/nd002/parts/0021345404/modules/316820862075461/lessons/5436095827/concepts/54446302850923#).
This function uses two Python mapping dictionaries that are specific to the particular problems in my map:

In [1]:
mapping = { "Ave": "Avenue", "Ave." : "Avenue", "Cres" : "Crescent", "Ct" : "Court", "Dr" : "Drive", 
            "Rd" : "Road","St": "Street"}

mapping2 = {"Rd E" : "Road East", "St E" : "Street East", "St N" : "Street North", "Blvd N" : "Boulevard North"}

These mapping dictionaries are used to transform street name abbreviations into the full street name with no abbreviations.  The code for ``` 'update_street_name()' ``` is given below, and is also contained in the ``` 'auditing_cleaning.py' ``` file.

In [2]:
import re

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)  # Python regular expression object that captures the last 
                                                          # word of a string                                   

# This function takes a street name and the two mapping dictionaries as input.  It will first strip extra whitespace
# from the street name (if there is any).  Then it checks if the last word of the street name is a key in the 'mapping'
# dictionary.  It also checks if the last two words of the street name is in the 'mapping2' dictionary.  If either of
# these conditions are met, then it fixes the street name according to the mappings in the respective dictionaries.

def update_street_name(street_name, mapping, mapping2):
    street_name = street_name.replace('  ', ' ')
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        double_street_type = ' '.join(street_name.split()[-2:])
        if street_type in mapping.keys():
            street_name = re.sub(street_type, mapping[street_type], street_name)
        if double_street_type in mapping2.keys():
            street_name = re.sub(double_street_type, mapping2[double_street_type], street_name)

    return street_name

### Postal Code Issues

Proper postal code formatting in Canada requires a six character string of the form 'A1A 1A1', where A is a letter and 1 is a digit.  There also must be a space separating the 3rd and 4th characters:  https://en.wikipedia.org/wiki/Postal_codes_in_Canada.  After auditing the postal codes (see the accompanying ``` 'auditing_cleaning.py' ``` file), I found that there were a few problems such as lower-case character issues (e.g. "l1c 2z2"), postal codes without the required whitespace (e.g. "L1H7K5"), or a mixture of both problems (e.g. "L1c2n4").  Also, there were two problematic postal codes ("l1c15e2" and "l1c") that I don't have enough information to fix, so I just left them.

Below is the function that I wrote to update the postal codes:

In [3]:
# This function takes a postal code and first converts it to upper-case characters (or does nothing otherwise).  
# If there is no whitespace, then it will check if the length of the postal code is 6.  If so, then it will add
# whitespace between the 3rd and 4th characters, as is required in the postal code formatting.

def update_postal_code(postal_code): 
    postal_code = postal_code.upper()
    if ' ' not in postal_code:
        if len(postal_code) == 6:
            postal_code = postal_code[0:3]+' '+postal_code[3:6]
            
    return postal_code

For example, we can run a test to see if this function updated the postal codes properly.  This test function can be found in the ``` 'auditing_cleaning.py' ``` file.  The test prints a list of postal codes both before and after they've been updated.  The first 10 results from this test are shown below:

```
l1c1t7 => L1C 1T7
l1c1p8 => L1C 1P8
l1c2l8 => L1C 2L8
l1c 2z2 => L1C 2Z2
l1c2n4 => L1C 2N4
l1c1p3 => L1C 1P3
l1c1p7 => L1C 1P7
l1c1t4 => L1C 1T4
l1c15e2 => L1C15E2
l1c1v8 => L1C 1V8
```

### City/Town Name Issues

My hometown is Bowmanville, Ontario (which is in Clarington), and I discovered that the town name is mispelled "Bowmanwille" in a few different places.  This was an easy fix; I just added the code:

```python
if city == 'Bowmanwille':
        city = city.replace('nw', 'nv')
```

to another function ``` 'update_city()' ``` that I wrote to update the city names (see the accompanying ``` 'auditing_cleaning.py' ``` file).

There were also some problems with city name consistency, e.g. sometimes the city Oshawa was referred to as "City of Oshawa".  Now I'm not really sure what the standard practice is in OpenStreetMap, but I wanted to have consistent queries when I uploaded the data to a database, so I opted to change all the occurences of "City of Oshawa" to simply "Oshawa".  Similarly, there were three different labels for the town of Whitby:  "Town of Whitby", "whitby", and "Whitby".  I chose to change all of these to just "Whitby".  The code for doing these things is straightforward and is also included in the ``` 'update_city()' ``` function in the accompanying ``` 'auditing_cleaning.py' ``` file.

### Way Tag with Missing House Number

After getting an error trying to validate my transformed data against the provided schema, Myles Callan @ Udacity (who is amazingly helpful) suggested that the culprit should be a missing value from one of the dictionaries in the ``` 'nodes_tags' ``` list.  To find this missing entry, I ran the code:

```python
for element in get_element(OSM_PATH, tags=('node', 'way')):
    for tag in element.iter('tag'):
        if tag.attrib['v'] == '' or tag.attrib['v'] == None:
            print tag.attrib
```

This code uses the Udacity-provided ``` 'get_element()' ``` function, and iterates over all the ``` 'tag' ``` tags of node and way elements.  If the ``` 'v' ``` attribute for a specific tag is the empty string or ``` None ```, then it prints the attribute dictionary for that tag.  The result was one tag that was the cause of the problem:

```python
{'k': 'addr:housenumber', 'v': ''}
```

The solution was to simply ignore this tag when processing the map.  It was a simple matter to add the required code to my ``` 'shape_element()' ``` function (see the accompanying ``` 'osm_to_csv.py' ``` file).

## Section 2:  Data Overview

This section contains a basic overview of the dataset and some information that I gathered by doing SQL queries on the corresponding database.  I chose to run SQL queries from within Python rather than the terminal, but only the actual SQL queries are included in this report (except for the first one).  See the accompanying ``` 'database_queries.py' ``` file for the full code.

### File Size

It's easier and faster to get the file size by just manually looking in Finder or doing it from the terminal, but I wanted to programmatically do it in Python (for kicks).

I used SQL ``` PRAGMA ``` statements ( https://www.sqlite.org/pragma.html#pragma_page_count ) to get the page_count and page_size for my database file ``` 'ClaringtonPlusDB.db' ```.  The file size is then the product of these two numbers.  Here is a way to programmatically do this in Python:

In [1]:
import sqlite3   # import the sqlite3 module in order to use SQLite within Python

database_file = 'ClaringtonPlusDB.db'
conn = sqlite3.connect(database_file)   # connect to the database
cursor = conn.cursor()                  # create a cursor object

# This function 'pragma()' will run an SQLite pragma statement along with a chosen command, and then return the result

def pragma(command):    
    query = "pragma {}".format(command)
    cursor.execute(query)
    results = cursor.fetchall()
    return results
    
    
page_size = float(pragma('page_size')[0][0])     # gives the page size in bytes
page_count = float(pragma('page_count')[0][0])   # gives the page count in bytes  
file_size_mb = (page_size * page_count) / 10**6  # converts the product of 'page_size' and 'page_count' to MB
print "File size:", file_size_mb

conn.close()

File size: 50.095104


So the database file is about 50.1 MB.  Also, the file size of the original ``` 'ClaringtonPlus.osm' ``` file is 86.5 MB.

### Number of Unique Users

To find this we must use both the ``` 'nodes' ``` table and the ``` 'ways' ``` table.  The following SQL query does the job:

```sql
SELECT count(sub.uid) FROM
(SELECT uid FROM nodes UNION SELECT uid FROM ways) sub;
```

The result is 186.

### Number of Nodes

```sql
SELECT count(*) FROM nodes;
```

The result is 357363

### Number of Ways

```sql
SELECT count(*) FROM ways;
```
The result is 47506

### Hamlets, Villages, Towns, Cities, etc.

The Municipality of Clarington contains a number of smaller villages, towns, hamlets, etc.  As mentioned above, the map I downloaded is larger than Clarington, so I expected it to contain even more places.  I was interested in getting a breakdown of the area by 'place' (e.g. town) and 'name' (e.g. Bowmanville).  The following query does the job:

```sql
SELECT sub.value, nt.value FROM nodes_tags nt JOIN 
(SELECT id, value FROM nodes_tags WHERE key = 'place') sub 
ON nt.id = sub.id WHERE nt.key = 'name'; 
```

The first 10 results are

```python
[(u'city', u'Oshawa'),
 (u'town', u'Port Perry'),
 (u'village', u'Burketon Station'),
 (u'town', u'Courtice'),
 (u'village', u'Pontypool'),
 (u'village', u'Kendal'),
 (u'village', u'Garden Hill'),
 (u'village', u'Tyrone'),
 (u'village', u'Enniskillen'),
 (u'village', u'Solina')]

```

There are some results here (Oshawa and Pontypool) that are not part of Clarington, but that was expected.  So what I am interested in now is the counts of each 'place', ie. how many villages, towns, cities, etc. are contained in my map.  The query is below:

```sql
SELECT value, count(*) FROM nodes_tags 
WHERE key = 'place' GROUP BY value;
```

The results are

```python
[(u'city', 1),
 (u'hamlet', 12),
 (u'locality', 11),
 (u'neighbourhood', 5),
 (u'town', 5),
 (u'village', 13)]
```

### Pizza Restaurants

This query returns a list of the pizza places in the map, along with a count of each one:

```sql
SELECT nt.value, count(*) as count FROM nodes_tags nt JOIN 
(SELECT id, value FROM nodes_tags WHERE value = 'pizza') sub 
ON nt.id = sub.id WHERE nt.key = 'name' GROUP BY nt.value ORDER BY count desc;
```

The results are

```python
[(u'Pizza Pizza', 9),
 (u'Pizza Hut', 2),
 (u"Beeno's Pizza", 1),
 (u'Boston Pizza', 1),
 (u'Free Topping Pizza', 1),
 (u'FreeTopping PIzza', 1),
 (u'Little Caesars Pizza', 1),
 (u'Pizzaville', 1),
 (u'Square Boy Pizza & Subs', 1),
 (u'Twice The Deal Pizza', 1),
 (u'Twice the Deal Pizza', 1)]
```

This is far from complete.  I can think of a number of pizza joints (Domino's, Al's Pizza, Rosalina's Pizza, Goodfellaspizzeria) that don't appear in this list.  Also, there is more than one Little Caesars and Square Boy.  This and some other problems will be discussed in the last section.

### Top 10 Users

This query uses both the nodes and ways table to find the top ten users (uid, name, and number of times they've made an edit):

```sql
SELECT sub.uid, sub.user, count(*) as count FROM
(SELECT uid, user FROM nodes UNION ALL SELECT uid, user FROM ways) sub 
GROUP BY sub.uid ORDER BY count desc LIMIT 10
```

The result is

```python
[(1679, u'andrewpmk', 313253),
 (51600, u'brandoncote', 34245),
 (155461, u'bdustan', 15933),
 (92209, u'geobase_stevens', 10640),
 (3551880, u'Mojgan Jadidi', 10344),
 (76077, u'Bootprint', 3320),
 (951370, u'fbax', 1233),
 (244083, u'timdine', 1074),
 (341519, u'lcmortensen', 1057),
 (5640, u'MikeyCarter', 1019)]
```

Wow, so it seems that this dude ``` 'andrewpmk' ``` is quite a prolific contributor, and his edits (313253) are about 9 times as much as the next user ``` 'brandoncote' ``` (34245 edits).

## Section 3:  Additional Ideas and Concluding Remarks

I think this dataset still needs quite a bit of work. There are still a number of opportunities for cleaning the data.  First, there are problems with non-standard phone number formatting, as the following query reveals:

```sql
SELECT sub.value FROM 
(SELECT key, value FROM nodes_tags UNION SELECT key, value FROM ways_tags) sub 
WHERE sub.key = 'phone'
```

There are several phone numbers formatted with dashes (e.g. 905-240-4424), several formatted without dashes (e.g. 905 697 8282), some with no space at all (e.g. 9054308400), and also other anomalies.  They should be formatted consistently and according to a set standard, probably this:  https://en.wikipedia.org/wiki/Telephone_numbers_in_Canada.  Fixing the phone numbers would probably not be very difficult, as it would just require writing another cleaning function in Python.

The next thing I would do is try to fix the 2 postal code issues that I mentioned above, ie. the 2 anomalies 'L1C' and 'L1C15E2'.  This could be done by digging deeper into the dataset, either with SQL queries, programmatically with Python, or just using "find" in textedit with the original OSM file.  Once the address or name is found for the offending postal codes, then we could simply check a database of postal codes in Ontario, Canada to find the right ones.

Another issue with the postal codes is there seems to be far too few of them.  This is likely because the dataset is incomplete.  We would need to get a list or database of postal codes for residences, businesses, etc. and then edit the OSM data.  This is something I may want to work on in the future, if I have time.

The same issue is true with pizza places, as mentioned above (and likely many other places as well).  There are a number of pizza places in my area that don't exist in the map.  These would probably need to be manually updated.

### Files Included

- ```auditing_cleaning.py``` - this file contains the Python auditing and cleaning functions.
- ```case_study_lesson6.py``` - contains the Python code for the Case Study exercises in Udacity's data wrangling course.
- ```ClaringtonPlus_sample.osm``` - an 8.7 MB sample of the original OSM file.
- ```database_queries.py``` - contains the SQL queries discussed in this report.
- ```MapLink_and_references``` - a text file containing a link to the map area above, and also a list of references.
- ```osm_to_csv.py``` - contains the ``` 'shape_element()' ``` function along with the other Python code for converting the OSM file to CSV files.